# 2. Training CLIP with MRL

This notebook demonstrates how to fine-tune a CLIP model using **Matryoshka Representation Learning (MRL)**.
Ensure you have run `01_setup_and_data.ipynb` first.

In [ ]:
import os

# Ensure we are in the project root
if os.path.exists("vembed-factory"):
    os.chdir("vembed-factory")
elif os.getcwd().endswith("notebooks"):
    os.chdir("..")

print(f"Working Directory: {os.getcwd()}")

In [ ]:
# Config Data Paths
if os.path.exists("data/flickr30k/train.jsonl"):
    DATA_PATH = "data/flickr30k/train.jsonl"
    IMAGE_ROOT = "data/flickr30k"
    VAL_DATA_PATH = "data/flickr30k/val.jsonl"
else:
    DATA_PATH = "data/dummy/train.jsonl"
    IMAGE_ROOT = "data/dummy"
    VAL_DATA_PATH = ""

## Start Training
We use `vembed/cli.py` (via `python -m vembed.cli`) to launch training.
- `model_type`: clip
- `use_mrl`: Enable Matryoshka Loss
- `mrl_dims`: [512, 256, 128] (Default for CLIP preset)

In [ ]:
!python -m vembed.cli \
    --model_type clip \
    --model_name "openai/clip-vit-base-patch32" \
    --data_path $DATA_PATH \
    --output_dir experiments/output_clip_mrl \
    --epochs 3 \
    --batch_size 32 \
    --use_mrl \
    --config_override image_root="$IMAGE_ROOT"

## Evaluation
We use the benchmark script to evaluate the MRL-trained model.

In [ ]:
if os.path.exists(VAL_DATA_PATH):
    # Evaluate on the validation set using MRL
    !python benchmark/run.py flickr30k \
        --model_path experiments/output_clip_mrl/checkpoint-epoch-3 \
        --flickr_root $IMAGE_ROOT \
        --output_dir experiments/eval_results_mrl \
        --batch_size 64 \
        --encoder_mode clip \
        --flickr_split val \
        --mrl_dims 512 256 128